# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from the Enron email dataset in a MongoDB database, and write a report detailing the suitability of different types of databases for data science applications.  You will be required to run code to answer the given questions in the Jupyter notebook provided, and write a report describing alternative approaches to using MongoDB.

Download the JSON version of the Enron data and import into a collection called messages in a database called enron.  You do not need to set up any authentication.  In the Jupyter notebook provided, perform the following tasks, using the Python PyMongo library.  Marks available for each question are enclosed in square brackets.

Answers should be efficient in terms of speed.  Answers which are less efficient will not get full marks.

In [1]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint

### 1)
Write a function which returns a MongoDB connection object to the "messages" collection. **[1]**

In [2]:
def get_collection():
    """
    Connects to the server, and returns a collection object
    of the `messages` collection in the `enron` database
    """
    # Assuming python is running in same machine, and mongo has default port
    client = MongoClient('mongodb://localhost:27017')

    return client.enron.messages

### 2)

Write a function which returns the amount of emails in the messages collection in total **[1]**

In [3]:
def get_amount_of_messages(collection):
    """
    :param collection A PyMongo collection object
    :return the amount of documents in the collection
    """    
    return collection.count()
    

### 3) 

Write a function which returns each person who was BCCed on an email.  Include each person only once, and display only their name according to the X-From header. **[2]**



In [4]:
def get_bcced_people(collection):
    """
    :param collection A PyMongo collection object
    :return the names of the people who have received an email by BCC
    """
    
    def helper_clean_up(results):
        """
        param: results - a list containing results to clean
        returns: a list of cleaned up results
        """
        
        # ASSUMING **PYTHON** REGEX MODULE IS NOT AVAILABLE
        # WOULD BE MUCH SIMPLER WITH IT, BUT CHALLENGE WAS STILL KIND OF FUN!!!!
        # RETURNS A COMPLETELY SEPARATED LIST OF ALL EMAIL ADDRESSES AND NAMES
        
        clean_email_results = []
        clean_name_results = []

        for result in results:

            # Assume not an email until evidence
            is_email = False
            # Assume not name until evidence
            is_name = False

            # Start with an empty variable
            building_result = ''

            # Extract email addresses from results with multiple recipients
            for i, char in enumerate(result + ','):

                # Ignore leading single quotes, commas and spaces
                if building_result == '' and ( char in ("'", ' ', ',') ):
                    continue

                # Build up the string
                building_result += char

                # If we find '@', assume it's an email address
                if char == '@':
                    is_email = True
                # If we're assuming we have an email, continue until we get to a comma or a single quote
                elif is_email and (char == ',' or char == "'"):
                    # Add result to list
                    clean_email_results.append(building_result[0:-1])
                    # Clear building_result
                    building_result = ''
                    # Again assume next item is not an email 
                    is_email = False
                    continue

                # If we find a comma, and we're not assuming an email
                elif not is_email and char == ',':
                    # Then if we are not assuming a name, start assuming a name
                    if not is_name:
                        is_name = True
                    # Capture the name
                    else:
                        # Add result to list
                        clean_name_results.append(building_result[0:-1])
                        # Clear building_result
                        building_result = ''
                        is_name = False
                        
            # Assume anything left in the builder at this point is a clean name with no comma
            clean_name_results.append(building_result[0:-1])

        # Deduplicate results 
        clean_set = set(clean_email_results + clean_name_results)
        
        # Remove any empty string record
        clean_set.remove('')

        # Convert to list and return
        return list(clean_set)
    
    
    # MAIN QUERY
    # Get messages with 'X-bcc' data, since 'X-bcc' is only bcc field with name info
    match_1 = {
        '$match' : {
            '$and': [
                {'headers.X-bcc': {'$exists': True }},
                {'headers.X-bcc': {'$ne': ''}}
            ]
        }
    }
    
    project_test = {
        '$project' : {
            "_id" : False,
            'bcc' : '$headers.X-bcc'
        }
    }
    
    # Split the data by '>, ' to get names with Microsoft Exchange Data, and unwind
    project_1 = {
        '$project' : {
            'split_exchange_1' : {
                '$split' : ["$headers.X-bcc", '>, ']
            }
        }
    }

    unwind_1 = {'$unwind': '$split_exchange_1'}

    # Split the data by ' <' and select first element to get rid of junk Microsoft Exchange Data
    project_2 = {
        '$project' : {
            '_id': False, 
            'split_exchange_2' : {
                '$arrayElemAt' : [{'$split' : ["$split_exchange_1", ' <']}, 0]
            }
        }
    }

    # Split the data by '', ' and unwind to separate out email addresses
    project_3 = {
        '$project' : {
            'bcc-name' : {
                '$split' : ["$split_exchange_2", "', "]
            }
        }
    }

    unwind_2 = {'$unwind': '$bcc-name'}

    # Split the data by '', ' and unwind to separate out email addresses
    project_3 = {
        '$project' : {
            'bcc-name' : {
                '$split' : ["$split_exchange_2", "', "]
            }
        }
    }

    unwind_2 = {'$unwind': '$bcc-name'}

    # Reduce number of duplicates before final processing with python
    group_1 = {
        '$group' : {
            '_id' :'$bcc-name'
        }
    }
    
    # Sort by total in decending order
    sort_test = {
        '$sort' :{
            '_id' : 1
        }
    }

    pipeline = [match_1, project_1, unwind_1, project_2, project_3, unwind_2, group_1, sort_test]
    
    cursor = list(collection.aggregate(pipeline))
    
    intermediate_results = [result['_id'] for result in cursor]
    
    return helper_clean_up(intermediate_results)

### 4)

Write a function with parameter subject, which gets all emails in a thread with that parameter, and orders them by date (ascending) **[3]**

In [5]:
def get_emails_in_thread(collection, subject):
    """
    :param collection A PyMongo collection object
    :return All emails in the thread with that subject
    """

    def regex_escape(input_string):
        # Process the input string (without 're' module) so it can be used in regex
        regex_special_chars = list('[\^$.|?*+()')
        pattern = ''

        for char in input_string:
            if char in regex_special_chars:
                pattern += '\\' + char
            else:
                pattern += char
        
        return pattern
            
    # Create a regex pattern to find the subject.
    # Include prefixes such as 'RE:' and 'FW:' to get all emails in thread
    # Case insensitive for prefixes for breadth and convenience
    # Case sensitive for subject parameter for accuracy
    subject_pattern =  '^((?i)(re|fwd|fw): ){1,}(?-i)' + regex_escape(subject) + '$'
    
    # Get messages with matching subject
    # Case sensitive chosen to match more accurately
    query_filter = {'headers.Subject': { '$regex': subject_pattern}}
    query_result = collection.find(filter=query_filter)
    
    # Sort using an anonymous function which extracts the date
    date_format = '%a, %d %b %Y %H:%M:%S %z'
    sorted_result = sorted(query_result, key = lambda k: datetime.strptime(k['headers']['Date'][0:-6], date_format))
    
    return sorted_result

### 5)

Write a function which returns the percentage of emails sent on a weekend (i.e., Saturday and Sunday) as a `float` between 0 and 1 **[3]**

In [6]:
def get_percentage_sent_on_weekend(collection):
    """
    :param collection A PyMongo collection object
    :return A float between 0 and 1
    """
    
    # The 'headers.Date' field always starts with a 3 char day code, but only Sat & Sun begin with 'S'
    # Could be done with one aggregation query, but collection.count() is cached DB statistic
    
    return collection.count(filter = {"headers.Date": {"$regex": '^S'}}) / collection.count()

### 6)

Write a function which takes an argument `email_address`, and `limit`.  The function should return the amount of emails sent, received, and total (sent + received) between each other email address, and the email address specified as an argument, as follows: `[{"contact": "michael.simmons@enron.com", "from": 42, "to": 92, "total": 134}, {"contact"......}]`. Use the `To`, `From`, and `Cc` headers.
* Sort the output by the total amount of emails descending
* The parameter limit which specifies how many results should be returned.  If it is null, the function should return them all. **[5]**

In [7]:
def get_emails_between_contacts(collection, email_address, limit):
    """
    Shows the communications between the person identified by `email_address`
    and all other contacts in the company sorted by the descending order
    of total emails using the To, From, and Cc headers.
    :param `collection` A PyMongo collection object    
    :param `limit` An integer specifying the amount to display, or
    if null will display all outputs
    :return A list of objects of the form:
    [{
        'contact': <<Another email address>>
        'from': 
        'to': 
        'total': 
    },{.....}]
    """

    def regex_escape(input_string):
        # Process the email address (without 're' module) so it can be used in regex
        regex_special_chars = list('[\^$.|?*+()')
        pattern = ''

        for char in input_string:
            if char in regex_special_chars:
                pattern += '\\' + char
            else:
                pattern += char
                
        return pattern
            
    email_pattern = regex_escape(email_address)

    # Match relevant messages
    match_1 = {
        '$match' : {
            '$or': [
                {'headers.From': email_address},
                {'headers.To': { '$regex' : email_pattern}},
                {'headers.Cc': { '$regex' : email_pattern}}
            ]
        }
    }

    # Determine sender and all recievers (To + Cc)
    # Also determine if sender was specified email_address
    project_1 = {
        '$project' : {
            #'_id' : False,
            'Sender' : '$headers.From',
            'Recievers' : {'$concat' : ["$headers.To", {'$ifNull' : [{'$concat' :[', ', '$headers.Cc']}, '']}]},
            'was_outgoing' : { '$eq': [ "$headers.From", email_address ] }
        }
    }

    # If specified email_address was a reciever, clear other irrelevant recievers
    project_2 = {
        '$project' : {
            'Sender' : True,
            'Recievers' : {'$cond' : ['$was_outgoing', "$Recievers", email_address]},
            'was_outgoing' : True
            }
    }

    # Split recievers by ", \r\n\t" delimiter and unwind
    project_3 = {
        '$project' : {
            'Sender' : True,
            'Recievers' : {'$split' : ["$Recievers", ", \r\n\t"]},
            'was_outgoing' : True
        }
    }

    unwind_1 = {'$unwind': '$Recievers'}

    # Split recievers by ", " delimiter and unwind
    project_4 = {
        '$project' : {
            'Sender' : True,
            'Recievers' : {'$split' : ["$Recievers", ", "]},
            'was_outgoing' : True
        }
    }

    unwind_2 = {'$unwind': '$Recievers'}
    
    # Remove any duplicate Recievers in To or Cc list for a single email
    group_1 = {
        '$group': {
            '_id': {
                'id': "$_id",
                'Reciever': "$Recievers",
                'Sender': '$Sender',
                'was_outgoing': '$was_outgoing'
            }
        }
    }

    # Determine the contact relative to the email_address, and increment the appropriate counts
    project_5 = {
        '$project' : {
            '_id' : False,
            'contact' : {'$cond' : ['$_id.was_outgoing', '$_id.Reciever', '$_id.Sender']},
            'from' : {'$cond' : ['$_id.was_outgoing', 0, 1]},
            'to' : {'$cond' : ['$_id.was_outgoing', 1, 0]},
        }
    }

    # Group by contact and sum counts
    group_2 = {
        '$group' : {
            '_id' :  '$contact',
            'from': { '$sum': "$from" },
            'to': { '$sum': "$to" },
            'total': { '$sum': 1 }
        }
    }

    # Rename fields
    project_6 = {
        '$project' : {
            '_id' : False,
            'contact' : '$_id',
            'from': True,
            'to' : True,
            'total' : True
        }
    }

    # Sort by total in decending order
    sort_1 = {
        '$sort' :{
            'total' : -1
        }
    }

    # Assemble the pipeline
    pipeline = [match_1, project_1, project_2, project_3, unwind_1, project_4,
                unwind_2, group_1, project_5, group_2, project_6, sort_1]

    if limit:
        limit = {'$limit' : limit}
        pipeline.append(limit)

    return list(collection.aggregate(pipeline))